In [75]:
import pandas as pd
import numpy

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

#Import data
all_colors = pd.read_csv("data/all-colors.csv")
chosen_colors = pd.read_csv("data/chosen-colors.csv")

#Remove chosen colors
condition = all_colors["Shirt Color Name"].isin(chosen_colors["Shirt Color"])
remaining_colors = all_colors.drop(all_colors[condition].index, inplace=False)

#Confirm removal
# print(len(remaining_colors))
# print(len(all_colors))
# print(len(chosen_colors))

In [98]:
import colormath.color_objects as co
import colormath.color_diff as cdiff

def compute_average_distance(row):
    color_Lab = None

    color_Lab = co.LabColor(*map(float, row["L*a*b Value"].split(", ")))

    distances = []

    #Compute all distances from chosen colors
    for index, row_remaining in chosen_colors.iterrows():
        other_color = co.LabColor(*map(float, row_remaining["L*a*b Value"].split(", ")))

        #Newer, asymmetric formula, weighted for fabrics, using
        distance = cdiff.delta_e_cmc(other_color, color_Lab , pl=2, pc=1)

        #Older symmetric formula, weighted for fabrics
        #distance = cdiff.delta_e_cie2000(color_Lab, other_color,Kl=2, Kc=1, Kh=1)

        distances += [distance]

    average_distance = sum(distances)/len(remaining_colors)

    row["Distance"] = average_distance
    return row

remaining_colors = remaining_colors.apply(compute_average_distance, axis=1)

In [99]:
sorted_remaining = remaining_colors.sort_values(by = "Distance", ascending=False)
sorted_remaining.iloc[:].head(10)

,Shirt Color Name,PMS Color Code,L*a*b Value,Hex,Distance
37,Poppy,2347C,"49.45, 75.12, 67.21",E10600,13.722151
24,Maize Yellow,7548CP,"83.14, 5.01, 85.02",F2CA00,12.989638
34,Orange,1645C,"65.69, 58.38, 58.65",FF6A39,11.935688
83,Heather Orange,1645 C,"65.69, 58.38, 58.65",FF6A39,11.935688
73,Heather Kelly,340 C,"51.80, -65.36, 14.55",00965E,11.553796
47,Teal,3262C,"66.44, -59.33, -12.05",00BFB2,11.106415
104,Heather Yellow,100 CP,"92.09, -6.83, 62.42",F6EB69,10.961603
9,Burnt Orange,159C,"54.43, 43.54, 58.62",CB6015,10.749025
95,Heather Red,193C,"42.79, 66.14, 27.76",BF0D3E,10.282757
105,Heather Yellow Gold,134 C,"86.67, 8.31, 54.63",FDD26E,9.848363
